# Image and text Generation


## Image Understanding

1. Embeddings
2. Decoding.

### 1. Embeddings

- Transfer learning = pretrained models on large dataset and apply that knowledge to smaller dataset.
- ResNet50 Model: That was trained on ImageNet dataset.
- For feature extraction use use the last hidden layers as numerical representation.
- There are two probelms with this approach. 
    -  The transfer learning (RestNet50) trained on the model only on images found in ImageNet.  But if there are differenttypes of images, diagrames, book pages, drawing, satellite image  the embeddings learned from the imagenet dataset may not works so well.
    - Also the labels may not be useful for specific classification tasks.


### Auxiliary Learning Tasks

- Another way to create embedding.
- A model is trained to predict the next work in sentence. 
- Auto encoders take advantage of an auxiliary learning task for images. 
- Similar to predict the next word model.

** Autoencoders**
- We take the image data, Pass it through a network that bottlenecks it into samllar internel vector.
- Then expands it back out into the dimensionality of the original image. 
- Any information in the inputs that isnt useful gets dropped.
- Input image ---> Encode --- latent dimension ---> Decoder ---> Reconstructed image.
- Ideal dimension size is tuned with expermiments. (Balanced not to lost important information and not to take too much noise)
- While restore the shape during decoding, The Conv2DTranspose - copies the value to its surrounding pixels.
- 


In [ ]:
### Encoder
encoder = tf.keras.Sequential([
    keras.Input(Shape=(28,28,1), name="image_input")
    layers.Conv2D(32, 3, activation="relu", strider=2, padding="same"),
    layers.Conv2d(64, 3, activation="relu", strides=2,  padding="same"),
    layers.Flatten(),
    layers.Dense(2) ] , name="encoder")

In [ ]:
### Decoder 
decoder = tf.kerasa.Sequential([
    keras.Input(shape=(latent_dim,), name ="d_input"),
    layers.Dense(7 * 7 * 64, activation="relu"),
    layers.Reshape((7, 7, 64)),
    layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same"),
    layers.Conv2DTranspose(1, 3, activation="sigmoid", strides=2, padding="same")
    ], name="decoder")

In [ ]:
### Training 
encoder_inputs = keras.Input(shape=(28,28,1))
x = encoder(encoder_inputs)
decoder_output = decoder(X)
autoencoder = keras.Model(encoder_inputs , decoder_output)

### Mean squeare error is caluclated between input and output images to calculate the loss function.
autoencoder.complie(optimizer=keras.optimizers.Adam(), loss='mse')
history = autoencoder.fit(mnist_digits, mnist_digits, epochs=30, batch_size=128)



Once the mode is trained, we can drop the decoder and use the ecoder to covnert images into latent vectors.
z = encoder.predict(img)

**Variational autoencoders (VAEs)** were developed in response to classic autoencoders being unable to use their decoders to generate quality images from user-generated latent vectors.

- VAE are generative models.
Generative models, on the other hand, learn a joint probability distribution that explicitly models the distribution of each class. The conditional probability distribution isn’t lost—we can still do classification using Bayes’ theorem, for example.

Fortunately, most of the architecture of variational autoencoders is the same as that of classic autoencoders: the hourglass shape, reconstruction loss, etc. However, the few additional complexities allow VAEs to do what autoencoders can’t: image generation.

**Implementing VAE**

In [ ]:
encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten(name="e_flatten")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)

In [ ]:
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()(z_mean, z_log_var)
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

### Sampling layer in TensofFlow

class Sampling(tf.keras.layers.Layer):
    def call(self, inputs)
        z_mean, z_log_var = inputs
        batch = tf.shape(input=z_mean)[0]
        dim = tf.shape(input=z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.math.exp(x=0.5 * z_log_var) * epsilon 
    

### VAE
z_mean, z_log_var, z = encoder(encoder_input)
decoder_ouput = decoder(z)
vae = keras.Model(encoder_inputs, decoder_output, name="vae")

### LOSS
# we cannot use the MSE
# KUllback-Liebler divargence - 

def kl_divergence(z_mean, z_log_var):
    kl_loss = 0.5 * (1 + z_log_var 
                     


history = vae.fit(mnist_digits, mnist_digits, epochs=30,
                  batch_size=128)